# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1N', 'insta_score': 0.967}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

Loaded lib dds.dll


[{'call': '4H', 'insta_score': 0.531, 'expected_score': 596, 'adjustment': 27},
 {'call': '3N', 'insta_score': 0.3, 'expected_score': 558, 'adjustment': 15}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx KTxxxx.xxx.K9x.x .AJTx.A8xxx.A98x J9xx.8x.Qxx.QTxx 0.95894 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12, 12.0, 12.3)  \n P-1N-P-3S-P-3N-P-P-P (10, 10.9, 11.3) ',
 'AQ8.KQ9x.JT.KJxx KJxx.Jxx.Axxx.8x .Axxx.KQ9xx.AT9x T9xxxx.T8.8x.Qxx 0.95832 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12, 12.0, 12.1)  \n P-1N-P-3S-P-3N-P-P-P (12, 9.9, 11.4) ',
 'AQ8.KQ9x.JT.KJxx KJTxx.A8x.98.xxx .Jxxx.AKQxx.QT8x 9xxxx.Tx.xxxx.A9 0.94991 \n P-1N-P-3S-P-4H-P-P-P (10, 10.9, 11.4)  \n P-1N-P-3S-P-3N-P-P-P (10, 8.4, 10.0) ',
 'AQ8.KQ9x.JT.KJxx Kxxxx.xxx.9x.Txx .AT8x.KQ8xx.AQ9x JT9xx.Jx.Axxx.8x 0.94387 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12, 12.3, 12.3)  \n P-1N-P-3S-P-3N-P-4C-P-4S-P-5C-P-5H-P-6N-P-P-P (12, 10.8, 11.8) ',
 'AQ8.KQ9x.JT.KJxx KJxx.T8xx.Kxxx.x .AJxx.A98xx.A98x T9xxxx.x.Qx.QTxx 0.94351 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12, 11.9, 12.1)  \n P-1N-P-3S-P-3N-P-P-P (11, 10.7, 11.3) ',
 'AQ8.KQ9x.JT.KJxx 9xxxx.Txx.Ax.Axx .AJ8x.KQ98x.Q98x KJTxx.xx.xxxx.Tx 0.94262 \n P-1N-P-3S-P-4H-P

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
AQ8.KQ9x.JT.KJxx KTxxxx.xxx.K9x.x .AJTx.A8xxx.A98x J9xx.8x.Qxx.QTxx
AQ8.KQ9x.JT.KJxx KJxx.Jxx.Axxx.8x .Axxx.KQ9xx.AT9x T9xxxx.T8.8x.Qxx
AQ8.KQ9x.JT.KJxx KJTxx.A8x.98.xxx .Jxxx.AKQxx.QT8x 9xxxx.Tx.xxxx.A9
AQ8.KQ9x.JT.KJxx Kxxxx.xxx.9x.Txx .AT8x.KQ8xx.AQ9x JT9xx.Jx.Axxx.8x
AQ8.KQ9x.JT.KJxx KJxx.T8xx.Kxxx.x .AJxx.A98xx.A98x T9xxxx.x.Qx.QTxx
AQ8.KQ9x.JT.KJxx 9xxxx.Txx.Ax.Axx .AJ8x.KQ98x.Q98x KJTxx.xx.xxxx.Tx
AQ8.KQ9x.JT.KJxx KJT9xx.xx.Qx.T8x .J8xx.AK8xx.AQxx xxxx.ATx.9xxx.9x
AQ8.KQ9x.JT.KJxx JT9xxx.Jx.Qxx.T8 .ATxx.AK8xx.AQxx Kxxx.8xx.9xx.9xx
AQ8.KQ9x.JT.KJxx JTxxx.T8.9xx.A9x .AJxx.AKQ8x.QTxx K9xxx.xxx.xxx.8x
AQ8.KQ9x.JT.KJxx Kxxxxx.xx.xx.ATx .AT8x.KQ8xx.Q98x JT9x.Jxx.A9xx.xx
AQ8.KQ9x.JT.KJxx KT9xx.JT.K8x.8xx .A8xx.AQxxx.AQxx Jxxxx.xxx.9xx.T9
AQ8.KQ9x.JT.KJxx T9xxxx.x.98x.QTx .AJ8x.AKQxx.A9xx KJxx.Txxx.xxx.8x
AQ8.KQ9x.JT.KJxx KTxxxx.8.8xxx.Qx .AJTx.AKQxx.ATxx J9xx.xxxx.9x.98x
AQ8.KQ9x.JT.KJxx KJTx.Axx.8xxx.98 .J8xx.AK9xx.QTxx 9xxxxx.Tx.Qx.Axx
AQ8.KQ9x.JT.KJxx KJxx.xx.Qxxx